In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from utils_projet import ConformalPrediction

In [3]:
df_train = pd.read_csv('datasets/dataset_part_1/spotify_dataset_train.csv')
df_test = pd.read_csv('datasets/dataset_part_1/spotify_dataset_test.csv')

In [4]:
# Séparation Features / Target 
X_train = df_train.select_dtypes(include=[np.number]).drop(columns=['duration_ms', 'key', 'mode'], errors='ignore')
y_train = df_train['genre']
X_test = df_test.select_dtypes(include=[np.number]).drop(columns=['duration_ms', 'key', 'mode'], errors='ignore')

# Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Utilisation des 1500 premiers exemples pour les tests.
N_SAMPLES = 1500
X_subset = X_train_scaled[:N_SAMPLES]
y_subset = y_train[:N_SAMPLES].values

# Entraînement du KNN de base
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_subset, y_subset)

# Création de l'objet ConformalPrediction
predictor = ConformalPrediction(knn, X_subset, y_subset)

In [5]:
# Choix d'un point de test
x_new_0 = X_test_scaled[0]

# Calcul des prédictions (p-values) pour le nouveau point
predictor.predict(x_new_0) 

# Génération des intervalles pour différents epsilons
eps_values = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

print(f"\n{'Epsilon':<10} | {'Confiance':<10} | {'Taille':<8} | {'Genres Prédits'}")
for eps in eps_values:
    # Calcul de l'intervalle
    interval = predictor.compute_interval(eps)
    confidence = (1 - eps) * 100
    print(f"{eps:<10} | {confidence:.0f}%{'':<9} | {len(interval):<8} | {interval}")

blues
chanson
classical
country
dance
disco
edm
electro
folk
hip hop
jazz
latin
metal
pop
punk
r&b
rap
reggae
rock
salsa
soul
techno

Epsilon    | Confiance  | Taille   | Genres Prédits
0.05       | 95%          | 14       | ['chanson', 'country', 'dance', 'edm', 'electro', 'folk', 'hip hop', 'jazz', 'pop', 'punk', 'r&b', 'rap', 'rock', 'soul']
0.1        | 90%          | 13       | ['chanson', 'country', 'dance', 'edm', 'electro', 'folk', 'hip hop', 'jazz', 'pop', 'r&b', 'rap', 'rock', 'soul']
0.2        | 80%          | 11       | ['chanson', 'country', 'dance', 'electro', 'folk', 'hip hop', 'pop', 'r&b', 'rap', 'rock', 'soul']
0.3        | 70%          | 6        | ['chanson', 'dance', 'pop', 'r&b', 'rap', 'rock']
0.4        | 60%          | 4        | ['chanson', 'pop', 'r&b', 'rock']
0.5        | 50%          | 3        | ['chanson', 'pop', 'r&b']


**Analyse des résultats :**

* **Q4 :** Pour $\epsilon=0.05$ (95% de confiance), l'intervalle contient beaucoup de genres (plus de 10). Le modèle est très incertain et doit inclure la majorité des classes pour garantir qu'il ne se trompe pas.
* **Q5 :** Lorsque l'on augmente $\epsilon$ comme 0.3 ou 0.5, notre niveau d'exigence diminue. L'intervalle se rétrécit et inclut moins de genres. Les genres restants sont très similaire sur le plan acoustique. Par exemple, "Pop", "Chanson" et "R&B" vont rester groupés, car le modèle élimine les genres diamétralement opposés comme le "Metal" ou le "Classique".

In [6]:
# # On teste 3 chansons différentes (ou plus) entre les indices 0 et 2832 pour comparer l'incertitude
indices_a_tester = [1, 1000, 2832]

for idx in indices_a_tester:
    x_new = X_test_scaled[idx]
    
    if idx != 0:
        predictor.predict(x_new)
        
    found_single = False
    for eps in np.arange(0.01, 1.0, 0.01):
        interval = predictor.compute_interval(eps)
        if len(interval) == 1:
            best_genre = interval[0]
            final_confidence = (1 - eps) * 100
            print(f"-> Genre isolé : '{best_genre}' | Confiance maximale : {final_confidence:.1f}% (eps={eps:.2f})")
            found_single = True
            break

blues
chanson
classical
country
dance
disco
edm
electro
folk
hip hop
jazz
latin
metal
pop
punk
r&b
rap
reggae
rock
salsa
soul
techno
-> Genre isolé : 'rock' | Confiance maximale : 57.0% (eps=0.43)
blues
chanson
classical
country
dance
disco
edm
electro
folk
hip hop
jazz
latin
metal
pop
punk
r&b
rap
reggae
rock
salsa
soul
techno
-> Genre isolé : 'dance' | Confiance maximale : 41.0% (eps=0.59)
blues
chanson
classical
country
dance
disco
edm
electro
folk
hip hop
jazz
latin
metal
pop
punk
r&b
rap
reggae
rock
salsa
soul
techno
-> Genre isolé : 'pop' | Confiance maximale : 13.0% (eps=0.87)


**Conclusion :**

En testant plusieurs chansons, on constate que l'incertitude pour obtenir une prédiction unique (un singleton) varie énormément d'un morceau à l'autre :
* Pour certaines chansons ambiguës, le modèle ne parvient à isoler un seul genre qu'avec une faible confiance. Le modèle KNN classique nous aurait sorti ce genre comme prédiction finale "Top 1", mais la prédiction conforme révèle qu'en réalité, ce n'était qu'un pari très risqué.
* Pour d'autres morceaux avec des caractéristiques plus tranchées comme le "métal", le modèle est capable d'isoler le genre avec une confiance beaucoup plus élevée.

**Bilan :** Cela démontre que la classification classique manque de nuances. La 'conformal prediction' offre une solution robuste en indiquant au système de recommandation quand il doit s'abstenir ou proposer des choix multiples plutôt que de faire une prédiction incertaine.